# Smets & Wouters (2007) - Replicación de Figura 4

Este notebook replica la **Figura 4** del paper "Shocks and Frictions in US Business Cycles: A Bayesian DSGE Approach" (AER, 2007).

**Figura 4**: Historical Decomposition of GDP Growth and Inflation
- Panel A: Annual per capita GDP growth (deviation from trend growth)
- Panel B: Inflation (deviation from mean)

La descomposición histórica muestra la contribución de cada shock estructural a los movimientos observados de estas variables durante el período de muestra (1966:Q1 - 2004:Q4).

**Nota sobre `mh_replic=0`:**
- Este parámetro NO afecta la descomposición histórica de shocks
- Solo significa que se usa el modo posterior (sin MCMC), lo cual es más rápido
- El smoother de Kalman funciona correctamente con el modo posterior

**Requisitos previos:**
- GNU Octave instalado y en PATH
- Dynare 6.x instalado
- Archivo `usmodel_mode.mat` con posterior mode

## 1. Setup y Configuración

In [ ]:
# Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import sys
from pathlib import Path

# Add parent directory to path
sys.path.append(str(Path.cwd().parent.parent))

from direct_replication import DynareInterface

# Configure plotting
plt.style.use('seaborn-v0_8-darkgrid')
%matplotlib inline

print("Imports completados")

In [ ]:
# Configurar rutas - MODIFICAR SEGUN TU INSTALACION

# Configurar ruta de Octave (necesario en Windows)
os.environ['OCTAVE_EXECUTABLE'] = r'C:\Program Files\GNU Octave\Octave-10.3.0\mingw64\bin\octave-cli.exe'

DYNARE_PATH = r'C:\dynare\6.5\matlab'  # Ruta a carpeta matlab de Dynare
REPO_PATH = Path.cwd().parent.parent / 'repo'  # Carpeta con datos (.xls)
MODEL_PATH = Path.cwd().parent / 'model'  # Carpeta con .mod files

print(f"Octave executable: {os.environ['OCTAVE_EXECUTABLE']}")
print(f"Dynare path: {DYNARE_PATH}")
print(f"Repo path: {REPO_PATH}")
print(f"Model path: {MODEL_PATH}")
print(f"Repo exists: {REPO_PATH.exists()}")
print(f"Model exists: {MODEL_PATH.exists()}")

### Test de Conexión Octave + Dynare

In [ ]:
from oct2py import Oct2Py

# Test basico
try:
    oc = Oct2Py()
    result = oc.eval('2 + 2', nout=1)
    print(f"Octave conectado: 2 + 2 = {result}")
    
    # Test Dynare
    oc.addpath(DYNARE_PATH)
    dynare_exists = oc.eval('exist("dynare")', nout=1)
    if dynare_exists == 2:
        print("Dynare encontrado")
    else:
        print("Dynare NO encontrado - verificar DYNARE_PATH")
    
    oc.exit()
    print("\nTodo listo!")
    
except Exception as e:
    print(f"Error: {e}")
    print("Ver setup_instructions.md para solucion de problemas")

## 2. Configuración para Figura 4

### Shocks estructurales del modelo:
- **Supply shocks:** Productivity (ea), Price markup (epinf), Wage markup (ew)
- **Demand shocks:** Risk premium (eb), Government spending (eg), Investment (eqs), Monetary (em)

In [ ]:
# Definiciones para Figura 4

# Labels para los 7 shocks estructurales
SHOCK_LABELS = {
    'ea': 'Productivity',
    'eb': 'Risk premium',
    'eg': 'Exog. spending',
    'eqs': 'Investment',
    'em': 'Monetary',
    'epinf': 'Price markup',
    'ew': 'Wage markup'
}

SHOCK_ORDER = ['ea', 'eb', 'eg', 'eqs', 'em', 'epinf', 'ew']

# Variables para Figura 4
FIGURE4_VARIABLES = ['dy', 'pinfobs']
FIGURE4_LABELS = {
    'dy': 'GDP Growth',
    'pinfobs': 'Inflation'
}

# Período de muestra
# first_obs=71 en usmodel = 1966:Q1 (dato 71 de 231 observaciones desde 1947:Q3)
START_YEAR = 1966
START_QUARTER = 1

# Colores para los shocks (consistente con otras figuras)
SHOCK_COLORS = {
    'ea': '#1f77b4',      # Azul - Productivity
    'eb': '#ff7f0e',      # Naranja - Risk premium
    'eg': '#2ca02c',      # Verde - Gov spending
    'eqs': '#d62728',     # Rojo - Investment
    'em': '#9467bd',      # Púrpura - Monetary
    'epinf': '#8c564b',   # Marrón - Price markup
    'ew': '#e377c2'       # Rosa - Wage markup
}

print("Configuración de Figura 4 definida")
print(f"\nVariables: {FIGURE4_VARIABLES}")
print(f"Shocks: {SHOCK_ORDER}")
print(f"Período: {START_YEAR}:Q{START_QUARTER} - 2004:Q4")

## 3. Ejecutar Dynare con Modelo para Figura 4

Usamos `usmodel_figure4.mod` que tiene:
- `smoother` en estimation para ejecutar el Kalman smoother
- `shock_decomposition` para calcular la descomposición histórica
- `mh_replic=0` para usar solo el modo posterior (sin MCMC)

In [ ]:
# Inicializar interfaz Dynare
di = DynareInterface(DYNARE_PATH, str(MODEL_PATH))

print("Interfaz Dynare inicializada")

In [ ]:
# Ejecutar modelo para Figura 4
# Este modelo incluye smoother y shock_decomposition

print("Ejecutando usmodel_figure4.mod...")
print("(Esto puede tardar varios minutos)\n")
print("Nota: mh_replic=0 significa que se usa el modo posterior sin MCMC.")
print("Esto NO afecta la descomposición histórica de shocks.\n")

di.run_model('usmodel_figure4.mod')

print("\nDynare completado")

## 4. Extraer Historical Shock Decomposition

La descomposición histórica muestra cómo cada shock estructural contribuyó a los movimientos observados de las variables durante el período de muestra.

In [ ]:
def extract_shock_decomposition_figure4(di, variables=['dy', 'pinfobs']):
    """
    Extrae la descomposición histórica de shocks de Dynare.
    
    En Dynare, oo_.shock_decomposition tiene dimensiones:
    (n_endo_vars x (n_shocks + 2) x n_periods)
    
    - Columnas 1 a n_shocks: contribución de cada shock
    - Columna n_shocks+1: contribución de condiciones iniciales
    - Columna n_shocks+2: variable suavizada (smoothed)
    
    Returns:
        DataFrame con columnas: variable, shock, period, year, quarter, contribution
    """
    # Verificar que existe shock_decomposition
    has_decomp = di.oc.eval('isfield(oo_, "shock_decomposition")', nout=1)
    if not has_decomp:
        raise RuntimeError("Shock decomposition no encontrada. Verificar que shock_decomposition se ejecutó.")
    
    # Obtener dimensiones
    decomp_size = di.oc.eval('size(oo_.shock_decomposition)', nout=1)
    decomp_size = np.array(decomp_size).flatten()
    n_vars = int(decomp_size[0])
    n_cols = int(decomp_size[1])
    n_periods = int(decomp_size[2])
    n_shocks = n_cols - 2  # Últimas 2 columnas son initial + smoothed
    
    print(f"Shock decomposition dimensions: {n_vars} vars x {n_cols} cols x {n_periods} periods")
    print(f"Number of shocks: {n_shocks}")
    print(f"Number of periods: {n_periods}")
    
    # Obtener nombres de variables endógenas (orden de declaración en var)
    endo_names_raw = di.oc.eval('M_.endo_names', nout=1)
    # Convertir a lista de strings
    if hasattr(endo_names_raw, 'tolist'):
        endo_names = [str(n).strip() for n in endo_names_raw.flatten()]
    else:
        endo_names = [str(n).strip() for n in endo_names_raw]
    
    # Nombres de shocks (orden de varexo)
    shock_names = ['ea', 'eb', 'eg', 'eqs', 'em', 'epinf', 'ew']
    
    print(f"\nEndogenous variables ({len(endo_names)}): {endo_names[:10]}...")
    print(f"Shocks: {shock_names}")
    
    # Encontrar índices de las variables de interés
    var_indices = {}
    for var in variables:
        try:
            idx = endo_names.index(var)
            var_indices[var] = idx
            print(f"Variable '{var}' encontrada en índice {idx}")
        except ValueError:
            print(f"WARNING: Variable '{var}' no encontrada en M_.endo_names")
    
    # Extraer array
    decomp_array = di.oc.eval('oo_.shock_decomposition', nout=1)
    print(f"\nShock decomposition array shape: {decomp_array.shape}")
    
    # Construir DataFrame
    data = []
    for var, v_idx in var_indices.items():
        for t in range(n_periods):
            # Calcular fecha
            total_quarters = (START_QUARTER - 1) + t
            year = START_YEAR + total_quarters // 4
            quarter = (total_quarters % 4) + 1
            
            # Contribución de cada shock
            for s_idx, shock in enumerate(shock_names):
                contribution = float(decomp_array[v_idx, s_idx, t])
                data.append({
                    'variable': var,
                    'shock': shock,
                    'period': t + 1,
                    'year': year,
                    'quarter': quarter,
                    'quarter_date': f"{year}Q{quarter}",
                    'contribution': contribution
                })
            
            # Condiciones iniciales (columna n_shocks)
            initial_contrib = float(decomp_array[v_idx, n_shocks, t])
            data.append({
                'variable': var,
                'shock': 'initial',
                'period': t + 1,
                'year': year,
                'quarter': quarter,
                'quarter_date': f"{year}Q{quarter}",
                'contribution': initial_contrib
            })
            
            # Variable suavizada (columna n_shocks + 1)
            smoothed_value = float(decomp_array[v_idx, n_shocks + 1, t])
            data.append({
                'variable': var,
                'shock': 'smoothed',
                'period': t + 1,
                'year': year,
                'quarter': quarter,
                'quarter_date': f"{year}Q{quarter}",
                'contribution': smoothed_value
            })
    
    return pd.DataFrame(data)

# Extraer shock decomposition
print("Extrayendo Historical Shock Decomposition...\n")
shock_decomp_df = extract_shock_decomposition_figure4(di, FIGURE4_VARIABLES)

print(f"\nShock decomposition extraída: {len(shock_decomp_df)} filas")
print(f"Variables: {shock_decomp_df['variable'].unique()}")
print(f"Shocks: {shock_decomp_df['shock'].unique()}")

In [ ]:
# Verificar que la suma de contribuciones = variable suavizada
print("Verificando consistencia de la descomposición...\n")

for var in FIGURE4_VARIABLES:
    var_data = shock_decomp_df[shock_decomp_df['variable'] == var]
    
    # Tomar algunos períodos de muestra
    for period in [1, 50, 100]:
        if period > var_data['period'].max():
            continue
            
        period_data = var_data[var_data['period'] == period]
        
        # Suma de shocks + initial
        shocks_sum = period_data[period_data['shock'].isin(SHOCK_ORDER + ['initial'])]['contribution'].sum()
        
        # Valor suavizado
        smoothed = period_data[period_data['shock'] == 'smoothed']['contribution'].values[0]
        
        diff = abs(shocks_sum - smoothed)
        status = "OK" if diff < 0.01 else "CHECK"
        
        quarter_date = period_data['quarter_date'].values[0]
        print(f"  {var} t={period} ({quarter_date}): Sum={shocks_sum:.4f}, Smoothed={smoothed:.4f}, Diff={diff:.6f} [{status}]")

print("\nVerificación completada")

In [ ]:
# Mostrar primeras filas
print("Primeras filas de la descomposición:")
shock_decomp_df.head(20)

## 5. Convertir GDP Growth Trimestral a Anual

La Figura 4 del paper muestra el crecimiento anual del PIB per cápita. Necesitamos convertir los datos trimestrales a anuales sumando las contribuciones de los 4 trimestres.

In [ ]:
def compute_annual_decomposition(shock_decomp_df, variable='dy'):
    """
    Convierte la descomposición trimestral a anual.
    
    Para tasas de crecimiento, el crecimiento anual es aproximadamente
    la suma de los 4 crecimientos trimestrales.
    
    Returns:
        DataFrame con descomposición anual
    """
    var_data = shock_decomp_df[shock_decomp_df['variable'] == variable].copy()
    
    # Agrupar por año y shock, sumando las contribuciones
    annual_data = var_data.groupby(['year', 'shock']).agg({
        'contribution': 'sum'
    }).reset_index()
    
    annual_data['variable'] = f'{variable}_annual'
    
    return annual_data

# Calcular descomposición anual para GDP growth
annual_gdp_decomp = compute_annual_decomposition(shock_decomp_df, 'dy')

print(f"Descomposición anual de GDP growth: {len(annual_gdp_decomp)} filas")
print(f"Años: {annual_gdp_decomp['year'].min()} - {annual_gdp_decomp['year'].max()}")

# Mostrar algunos años
print("\nEjemplo para 1975 (recesión):")
annual_gdp_decomp[annual_gdp_decomp['year'] == 1975]

## 6. Calcular Desviaciones de la Media

El paper muestra las desviaciones respecto a la tendencia/media. Para el modelo linealizado, las variables ya están en desviación del estado estacionario.

In [ ]:
# Verificar la media de las variables suavizadas
print("Estadísticas de las variables suavizadas (deviation from steady state):\n")

for var in FIGURE4_VARIABLES:
    smoothed_data = shock_decomp_df[
        (shock_decomp_df['variable'] == var) & 
        (shock_decomp_df['shock'] == 'smoothed')
    ]['contribution']
    
    print(f"{FIGURE4_LABELS[var]}:")
    print(f"  Mean: {smoothed_data.mean():.4f}")
    print(f"  Std:  {smoothed_data.std():.4f}")
    print(f"  Min:  {smoothed_data.min():.4f}")
    print(f"  Max:  {smoothed_data.max():.4f}")
    print()

## 7. Graficar Figura 4: Historical Decomposition

Creamos gráficos de áreas apiladas mostrando la contribución de cada shock a los movimientos históricos de GDP growth e Inflation.

In [ ]:
def plot_historical_decomposition(decomp_df, variable, shock_order, shock_labels, shock_colors,
                                   title, include_initial=True, ax=None):
    """
    Grafica la descomposición histórica como gráfico de áreas apiladas.
    
    Args:
        decomp_df: DataFrame con la descomposición
        variable: Variable a graficar
        shock_order: Orden de los shocks
        shock_labels: Labels para los shocks
        shock_colors: Colores para los shocks
        title: Título del gráfico
        include_initial: Si incluir condiciones iniciales
        ax: Matplotlib axes (opcional)
    """
    if ax is None:
        fig, ax = plt.subplots(figsize=(14, 6))
    
    # Filtrar datos para esta variable
    var_data = decomp_df[decomp_df['variable'] == variable].copy()
    
    # Obtener períodos únicos ordenados
    periods = sorted(var_data['period'].unique())
    
    # Crear eje X con fechas
    dates = []
    for p in periods:
        p_data = var_data[var_data['period'] == p].iloc[0]
        dates.append(p_data['year'] + (p_data['quarter'] - 1) / 4)
    dates = np.array(dates)
    
    # Preparar datos para stacked area
    shocks_to_plot = shock_order.copy()
    if include_initial:
        shocks_to_plot.append('initial')
    
    # Separar contribuciones positivas y negativas
    # Para un stacked area apropiado, necesitamos manejar valores negativos
    
    # Obtener matriz de contribuciones
    contrib_matrix = np.zeros((len(shocks_to_plot), len(periods)))
    for s_idx, shock in enumerate(shocks_to_plot):
        for p_idx, period in enumerate(periods):
            val = var_data[(var_data['period'] == period) & (var_data['shock'] == shock)]['contribution']
            if len(val) > 0:
                contrib_matrix[s_idx, p_idx] = val.values[0]
    
    # Colores para los shocks
    colors = [shock_colors.get(s, '#888888') for s in shocks_to_plot]
    labels = [shock_labels.get(s, s) for s in shocks_to_plot]
    if include_initial:
        labels[-1] = 'Initial conditions'
    
    # Plot stacked bars (mejor para valores positivos y negativos mezclados)
    bar_width = 0.8 * (dates[1] - dates[0]) if len(dates) > 1 else 0.2
    
    # Apilar las contribuciones
    bottom_pos = np.zeros(len(periods))
    bottom_neg = np.zeros(len(periods))
    
    for s_idx, shock in enumerate(shocks_to_plot):
        values = contrib_matrix[s_idx, :]
        
        # Separar positivos y negativos
        pos_vals = np.maximum(values, 0)
        neg_vals = np.minimum(values, 0)
        
        # Graficar positivos
        ax.bar(dates, pos_vals, bar_width, bottom=bottom_pos, 
               color=colors[s_idx], label=labels[s_idx], alpha=0.8, edgecolor='none')
        bottom_pos += pos_vals
        
        # Graficar negativos
        ax.bar(dates, neg_vals, bar_width, bottom=bottom_neg,
               color=colors[s_idx], alpha=0.8, edgecolor='none')
        bottom_neg += neg_vals
    
    # Línea de la variable suavizada
    smoothed_data = var_data[var_data['shock'] == 'smoothed'].sort_values('period')
    ax.plot(dates, smoothed_data['contribution'].values, 'k-', 
            linewidth=1.5, label='Actual', zorder=10)
    
    # Línea de referencia en cero
    ax.axhline(0, color='black', linewidth=0.8)
    
    # Formato
    ax.set_title(title, fontsize=12, fontweight='bold')
    ax.set_xlabel('Year', fontsize=10)
    ax.set_ylabel('Percent', fontsize=10)
    ax.set_xlim(dates[0] - 0.5, dates[-1] + 0.5)
    ax.grid(True, alpha=0.3, linestyle='--')
    
    # Leyenda
    ax.legend(loc='upper left', fontsize=8, ncol=4, framealpha=0.9)
    
    return ax

In [ ]:
# Generar Figura 4 completa
print("Generando Figura 4: Historical Decomposition...\n")

fig, axes = plt.subplots(2, 1, figsize=(14, 10))

# Panel A: GDP Growth (trimestral, en desviación)
plot_historical_decomposition(
    shock_decomp_df,
    'dy',
    SHOCK_ORDER,
    SHOCK_LABELS,
    SHOCK_COLORS,
    'A. GDP Growth (quarterly, deviation from trend)',
    include_initial=True,
    ax=axes[0]
)

# Panel B: Inflation
plot_historical_decomposition(
    shock_decomp_df,
    'pinfobs',
    SHOCK_ORDER,
    SHOCK_LABELS,
    SHOCK_COLORS,
    'B. Inflation (deviation from mean)',
    include_initial=True,
    ax=axes[1]
)

plt.suptitle('Figure 4: Historical Decomposition of GDP Growth and Inflation\n(Smets & Wouters 2007)', 
             fontsize=14, fontweight='bold')
plt.tight_layout(rect=[0, 0, 1, 0.96])

fig4 = fig
plt.show()

print("\nFigura 4 generada")

## 8. Versión Alternativa: Descomposición Anual de GDP

In [ ]:
def plot_annual_decomposition(annual_decomp, shock_order, shock_labels, shock_colors, title):
    """
    Grafica la descomposición anual de GDP growth.
    """
    fig, ax = plt.subplots(figsize=(14, 6))
    
    years = sorted(annual_decomp['year'].unique())
    
    shocks_to_plot = shock_order + ['initial']
    colors = [shock_colors.get(s, '#888888') for s in shocks_to_plot]
    labels = [shock_labels.get(s, s) for s in shocks_to_plot]
    labels[-1] = 'Initial conditions'
    
    # Preparar matriz
    contrib_matrix = np.zeros((len(shocks_to_plot), len(years)))
    for s_idx, shock in enumerate(shocks_to_plot):
        for y_idx, year in enumerate(years):
            val = annual_decomp[(annual_decomp['year'] == year) & (annual_decomp['shock'] == shock)]['contribution']
            if len(val) > 0:
                contrib_matrix[s_idx, y_idx] = val.values[0]
    
    bar_width = 0.7
    x_pos = np.arange(len(years))
    
    bottom_pos = np.zeros(len(years))
    bottom_neg = np.zeros(len(years))
    
    for s_idx, shock in enumerate(shocks_to_plot):
        values = contrib_matrix[s_idx, :]
        pos_vals = np.maximum(values, 0)
        neg_vals = np.minimum(values, 0)
        
        ax.bar(x_pos, pos_vals, bar_width, bottom=bottom_pos,
               color=colors[s_idx], label=labels[s_idx], alpha=0.8)
        bottom_pos += pos_vals
        
        ax.bar(x_pos, neg_vals, bar_width, bottom=bottom_neg,
               color=colors[s_idx], alpha=0.8)
        bottom_neg += neg_vals
    
    # Línea de la suma (actual)
    total = annual_decomp[annual_decomp['shock'] == 'smoothed'].sort_values('year')['contribution'].values
    if len(total) == len(years):
        ax.plot(x_pos, total, 'k-o', linewidth=2, markersize=4, label='Actual', zorder=10)
    
    ax.axhline(0, color='black', linewidth=0.8)
    ax.set_title(title, fontsize=12, fontweight='bold')
    ax.set_xlabel('Year', fontsize=10)
    ax.set_ylabel('Percent (annual)', fontsize=10)
    ax.set_xticks(x_pos[::2])  # Cada 2 años
    ax.set_xticklabels([str(y) for y in years[::2]], rotation=45)
    ax.legend(loc='upper left', fontsize=8, ncol=4)
    ax.grid(True, alpha=0.3, linestyle='--')
    
    plt.tight_layout()
    return fig

# Graficar versión anual
print("Generando descomposición anual de GDP Growth...\n")

fig_annual = plot_annual_decomposition(
    annual_gdp_decomp,
    SHOCK_ORDER,
    SHOCK_LABELS,
    SHOCK_COLORS,
    'Figure 4A (Annual): Historical Decomposition of Annual GDP Growth'
)

plt.show()
print("\nDescomposición anual generada")

## 9. Exportar Resultados

In [ ]:
# Guardar figuras
output_dir = Path.cwd()

fig4.savefig(output_dir / 'figure4_historical_decomposition.png', dpi=300, bbox_inches='tight')
print(f"Figura 4 guardada en: {output_dir / 'figure4_historical_decomposition.png'}")

fig_annual.savefig(output_dir / 'figure4_annual_gdp_decomposition.png', dpi=300, bbox_inches='tight')
print(f"Figura 4 (anual) guardada en: {output_dir / 'figure4_annual_gdp_decomposition.png'}")

# Guardar datos en CSV
shock_decomp_df.to_csv(output_dir / 'figure4_shock_decomposition_data.csv', index=False)
print(f"\nDatos descomposición guardados en: {output_dir / 'figure4_shock_decomposition_data.csv'}")

annual_gdp_decomp.to_csv(output_dir / 'figure4_annual_gdp_decomposition_data.csv', index=False)
print(f"Datos anuales guardados en: {output_dir / 'figure4_annual_gdp_decomposition_data.csv'}")

## 10. Resumen de Resultados

In [ ]:
# Resumen de contribuciones por shock
print("="*70)
print("RESUMEN: Contribución promedio de cada shock")
print("="*70)

for var in FIGURE4_VARIABLES:
    print(f"\n{FIGURE4_LABELS[var]}:")
    print("-" * 50)
    
    var_data = shock_decomp_df[
        (shock_decomp_df['variable'] == var) & 
        (shock_decomp_df['shock'].isin(SHOCK_ORDER))
    ]
    
    # Estadísticas por shock
    for shock in SHOCK_ORDER:
        shock_data = var_data[var_data['shock'] == shock]['contribution']
        print(f"  {SHOCK_LABELS[shock]:15s}: Mean={shock_data.mean():+.3f}, Std={shock_data.std():.3f}")

In [ ]:
# Eventos históricos clave
print("\n" + "="*70)
print("EVENTOS HISTORICOS CLAVE")
print("="*70)

key_periods = {
    '1974-1975 (Oil Crisis/Recession)': (1974, 1975),
    '1980-1982 (Volcker Disinflation)': (1980, 1982),
    '1990-1991 (Gulf War Recession)': (1990, 1991),
    '2001 (Dot-com Recession)': (2001, 2001)
}

for event_name, (start_year, end_year) in key_periods.items():
    print(f"\n{event_name}:")
    print("-" * 50)
    
    for var in FIGURE4_VARIABLES:
        var_data = shock_decomp_df[
            (shock_decomp_df['variable'] == var) & 
            (shock_decomp_df['year'] >= start_year) &
            (shock_decomp_df['year'] <= end_year)
        ]
        
        print(f"  {FIGURE4_LABELS[var]}:")
        
        # Top 3 shocks contribuyendo
        shock_means = var_data[var_data['shock'].isin(SHOCK_ORDER)].groupby('shock')['contribution'].mean()
        top_shocks = shock_means.abs().nlargest(3)
        
        for shock in top_shocks.index:
            val = shock_means[shock]
            print(f"    {SHOCK_LABELS[shock]:15s}: {val:+.3f}")

In [ ]:
print("\n" + "="*70)
print("REPLICACION FIGURA 4 COMPLETADA")
print("="*70)
print("\nFiguras generadas:")
print("  - figure4_historical_decomposition.png: Descomposición histórica (trimestral)")
print("  - figure4_annual_gdp_decomposition.png: Descomposición GDP anual")
print("\nDatos exportados:")
print("  - figure4_shock_decomposition_data.csv")
print("  - figure4_annual_gdp_decomposition_data.csv")
print("\nNota: mh_replic=0 fue utilizado (modo posterior sin MCMC).")
print("Esto NO afecta la descomposición histórica de shocks.")

## 11. Cleanup

In [ ]:
# Cerrar sesión Octave
di.close()
print("Sesión Octave cerrada")